# Atomic Forces and Geometry Optimizaiton

## What this tutorial is about 

In this tutorial we will learn how to compute the excited state atomic forces of a molecular structure and how to use these to perform geometry optimization. Let's start by importing the relevant modules

In [ ]:
from pyxtp import xtp
from ase.io import write
from ase.build import molecule
from ase.optimize import GoodOldQuasiNewton

## Define the molecular structure

We define here a `CO` molecule in its ground state geometry.  

In [ ]:
atoms = molecule('CO')

## Configure the `xtp` calculator

We now define the `xtp` calculator and configure it to target particular forces. We can select which forces we want to compute with the method `.select_force()` of the calculator. We can choose to target different forces :
* `energy='energy'` : DFT total energy
* `energy='singlets'`: BSE singlet energy
* `energy='triplets'`: BSE triplet energy 
* `energy='qp'`: Quasi particile energy

For `singlets`, `triplets` and `qp` one can also specify which particular level to target to compute the forces. For example `energy='singlets', level=0` will target the lowest lying singlet state. 

In [ ]:
# instantiate the calculator
calc = xtp(nthreads=2)

# select the force we want to use
calc.select_force(energy='singlets', level=0, dynamic=False)

# this allows to change all options
# calc.options.dftpackage.functional = 'PBE'
calc.options.dftpackage.basisset = 'def2-svp'
calc.options.dftpackage.auxbasisset = 'aux-def2-svp'

# set up the logger
calc.options.logging_file = 'CO_forces.log'

# set the calculator
atoms.calc = calc

## Compute the forces

If you are simply interested in computing the forces, they can easily be accessed through the `.get_forces()` method

In [ ]:
atoms.get_forces()

## Geometry optimization

Geometry optimization can be run by leveraging the intrinsic ASE capabilities. We can for example use the QuasiNewton method implemented in ASE to relax the molecular structre in the excited states we have just specified. We here fix the number of steps to 10 to limit the computational cost. 

In [ ]:
dyn = GoodOldQuasiNewton(atoms, trajectory='test.traj')
dyn.run(fmax=0.05, steps=10)
write('final.xyz', atoms)